In [116]:
import pandas as pd
import requests
import mysql.connector
import os
import numpy as np

In [128]:
#Baixar dados do github
# tabelas = [
#     "https://raw.githubusercontent.com/ikeisuke-ando/Explorando-problemas-reais-utilizando-Business-Intelligence/main/tabelas-usadas/Tabela%202.1%20(RendCaract_Geo).xls",
#     "https://raw.githubusercontent.com/ikeisuke-ando/Explorando-problemas-reais-utilizando-Business-Intelligence/main/tabelas-usadas/Tabela%202.2%20(RendCaract_Geo_sbenef).xls",
#     "https://raw.githubusercontent.com/ikeisuke-ando/Explorando-problemas-reais-utilizando-Business-Intelligence/main/tabelas-usadas/indicadores_trajetoria_educacao_superior_2020_2024.xlsx"
# ]
# 
# for tabela in tabelas:
#     nome = tabela.split("/")[-1]
#     if not os.path.exists(nome):
#         r = requests.get(tabela)
#         with open(nome, "wb") as f:
#             f.write(r.content)

df_acad = pd.read_excel( "tabelas_alteradas/indicadores_trajetoria_educacao_superior.xlsx")
df_renda = pd.read_excel( "tabelas_alteradas/rendimento_domiciliar_per_capita.xlsx")

In [ ]:
print(df_acad)

In [ ]:
print(df_renda)

In [126]:
# Conectando com o mysql
conn = mysql.connector.connect(
    host="localhost",
    user="root", 
    password="0000",
)
cursor = conn.cursor()

cursor.execute("CREATE DATABASE IF NOT EXISTS datawarehouse") # Criando Banco
conn = mysql.connector.connect(
    host="localhost",
    user="root",
    password="0000",
    database="datawarehouse" 
)
cursor = conn.cursor()

In [113]:
# Criando tabelas caso não exista no banco
tabelas_sql = [
    """
    CREATE TABLE IF NOT EXISTS `cursos` (
        `cur_id` int NOT NULL,
        `cur_cod` int DEFAULT NULL,
        `cur_nome` varchar(255) DEFAULT NULL,
        `cur_cod_area` varchar(20) DEFAULT NULL,
        `cur_nome_area` varchar(255) DEFAULT NULL,
        `cur_modalidade_ensino` varchar(100) DEFAULT NULL,
        PRIMARY KEY (`cur_id`)
    )
    """,
    """
    CREATE TABLE IF NOT EXISTS `tempo` (
        `tem_id` int unsigned NOT NULL AUTO_INCREMENT,
        `tem_ano` int NOT NULL,
        PRIMARY KEY (`tem_id`)
    )
    """,
    """ 
    CREATE TABLE IF NOT EXISTS `uf` (
        `uf_id` int unsigned NOT NULL,
        `uf_nome` varchar(255) NOT NULL DEFAULT '',
        `uf_sigla` char(2) NOT NULL DEFAULT '',
        PRIMARY KEY (`uf_id`)
    )
    """,
    """
    CREATE TABLE IF NOT EXISTS `universidades` (
        `uni_id` int NOT NULL,
        `uni_nome` varchar(255) DEFAULT NULL,
        PRIMARY KEY (`uni_id`)
    )
    """,
    """
    CREATE TABLE IF NOT EXISTS `indicadores_academicos` (
        `uni_id` int NOT NULL,
        `cur_id` int NOT NULL,
        `uf_id` int unsigned NOT NULL,
        `tem_id` int unsigned NOT NULL,
        `ano_ingresso` int NOT NULL,
        `prazo_integracao_anos` int NOT NULL,
        `ano_integracao_curso` int NOT NULL,
        `prazo_acompanhamento_anos` int NOT NULL,
        `ano_maximo_acompanhamento` int NOT NULL,
        `qt_ingressantes` int DEFAULT '0',
        `qt_permanencia` int DEFAULT '0',
        `qt_concluintes` int DEFAULT '0',
        `qt_desistencia` int DEFAULT '0',
        `qt_falecidos` int DEFAULT '0',
        `taxa_permanencia` float DEFAULT '0',
        `taxa_conclusao_acumulada` float DEFAULT '0',
        `taxa_desistencia_acumulada` float DEFAULT '0',
        `taxa_conclusao_anual` float DEFAULT '0',
        `taxa_desistencia_anual` float DEFAULT '0',
        PRIMARY KEY (`uni_id`,`cur_id`,`uf_id`,`tem_id`),
        KEY `idicadores_academicos_fk_3` (`uf_id`),
        KEY `indicadores_academicos_fk_2` (`cur_id`),
        KEY `indicadores_academicos_fk_3` (`tem_id`),
        CONSTRAINT `idicadores_academicos_fk_3` FOREIGN KEY (`uf_id`) REFERENCES `uf` (`uf_id`),
        CONSTRAINT `indicadores_academicos_fk_1` FOREIGN KEY (`uni_id`) REFERENCES `universidades` (`uni_id`),
        CONSTRAINT `indicadores_academicos_fk_2` FOREIGN KEY (`cur_id`) REFERENCES `cursos` (`cur_id`),
          CONSTRAINT `indicadores_academicos_fk_3` FOREIGN KEY (`tem_id`) REFERENCES `tempo` (`tem_id`)
    )
    """,
    """
    CREATE TABLE IF NOT EXISTS `indicadores_economicos` (
        `uf_id` int unsigned NOT NULL,
        `tem_id` int unsigned NOT NULL,
        `inec_rendimento_medio_percapita_sem_beneficio` float DEFAULT '0',
        `inec_rendimento_medio_percapita_com_beneficio` float DEFAULT '0',
        `inec_coeficiente_variacao_rendimento_com_beneficio` float DEFAULT '0',
        `inec_coeficiente_variacao_rendimento_sem_beneficio` float DEFAULT '0',
        KEY `indicadoreseconomicos_fk_1` (`uf_id`),
        KEY `indicadores_economicos_fk_2` (`tem_id`),
        CONSTRAINT `indicadores_economicos_fk_2` FOREIGN KEY (`tem_id`) REFERENCES `tempo` (`tem_id`),
        CONSTRAINT `indicadoreseconomicos_fk_1` FOREIGN KEY (`uf_id`) REFERENCES `uf` (`uf_id`)
    )
    """,   
]

for sql in tabelas_sql:
    cursor.execute(sql)

In [129]:
# Inserir dados no banco

# Setando dados dos estados 

cursor.execute("""
    INSERT IGNORE INTO uf (uf_id, uf_nome, uf_sigla) VALUES
    (11, 'Rondônia', 'RO'),
    (12, 'Acre', 'AC'),
    (13, 'Amazonas', 'AM'),
    (14, 'Roraima', 'RR'),
    (15, 'Pará', 'PA'),
    (16, 'Amapá', 'AP'),
    (17, 'Tocantins', 'TO'),
    (21, 'Maranhão', 'MA'),
    (22, 'Piauí', 'PI'),
    (23, 'Ceará', 'CE'),
    (24, 'Rio Grande do Norte', 'RN'),
    (25, 'Paraíba', 'PB'),
    (26, 'Pernambuco', 'PE'),
    (27, 'Alagoas', 'AL'),
    (28, 'Sergipe', 'SE'),
    (29, 'Bahia', 'BA'),
    (31, 'Minas Gerais', 'MG'),
    (32, 'Espírito Santo', 'ES'),
    (33, 'Rio de Janeiro', 'RJ'),
    (35, 'São Paulo', 'SP'),
    (41, 'Paraná', 'PR'),
    (42, 'Santa Catarina', 'SC'),
    (43, 'Rio Grande do Sul', 'RS'),
    (50, 'Mato Grosso do Sul', 'MS'),
    (51, 'Mato Grosso', 'MT'),
    (52, 'Goiás', 'GO'),
    (53, 'Distrito Federal', 'DF');
""")

# Setando dados do tempo

cursor.execute("""
    INSERT IGNORE INTO tempo (tem_id, tem_ano) VALUES
    (1, 2020),
    (2, 2021),
    (3, 2022),
    (4, 2023),
    (5, 2024),
    (6, 2025);
""")

# Formatando dados para inserir na tabela indicadores economicos
dados = []
cursor.execute("SELECT tem_id, tem_ano FROM tempo;")
mapa_tempo = {ano: tem_id for tem_id, ano in cursor.fetchall()}


for i, row in df_renda.iterrows():
    uf_id = row["regioes"]
    
    for ano in [2020, 2021, 2022, 2023]:
        dados.append({
            "uf_id": uf_id,
            "ano": ano,
            "media_com_beneficios": row[f"media_sem_beneficios_{ano}"],
            "media_sem_beneficios": row[f"media_sem_beneficios_{ano}"],
            "coeficiente_variacao_com_beneficios": row[f"media_cv_com_beneficios_{ano}"],
            "coeficiente_variacao_sem_beneficios": row[f"media_cv_sem_beneficios_{ano}"]
        })


df_preparado = pd.DataFrame(dados)

valores = []

for i, row in df_preparado.iterrows():
    tem_id = mapa_tempo[row["ano"]]
    valores.append((
        int(row["uf_id"]),
        int(tem_id),
        row["media_com_beneficios"],
        row["media_sem_beneficios"],
        row["coeficiente_variacao_com_beneficios"],
        row["coeficiente_variacao_sem_beneficios"]
    ))
    
sql = """
    INSERT INTO indicadores_economicos (
        uf_id, tem_id,
        inec_rendimento_medio_percapita_com_beneficio,
        inec_rendimento_medio_percapita_sem_beneficio,
        inec_coeficiente_variacao_rendimento_com_beneficio,
        inec_coeficiente_variacao_rendimento_sem_beneficio
    )
    VALUES (%s, %s, %s, %s, %s, %s)
"""

cursor.executemany(sql, valores)

# Tratando dados academicos

universidades = []
# Usando um set para evitar duplicatas
seen_unis = set()

for i, row in df_acad.iterrows():
    uni_id = int(row["Código da Instituição"])
    uni_nome = row["Nome da Instituição"]
    if uni_id not in seen_unis:
        universidades.append((uni_id, uni_nome))
        seen_unis.add(uni_id)

sql_uni = """
INSERT INTO universidades (uni_id, uni_nome) VALUES (%s, %s)
ON DUPLICATE KEY UPDATE uni_nome = VALUES(uni_nome);
"""
cursor.executemany(sql_uni, universidades)

cursos = []
seen_cursos = set()

for i, row in df_acad.iterrows():
    cur_id = int(row["Código do Curso de Graduação"])
    cur_nome = row["Nome do Curso de Graduação"]
    cur_cod_area  = row.get("Código da área do Curso segundo a classificação CINE BRASIL", "")
    cur_nome_area = row.get("Nome da área do Curso segundo a classificação CINE BRASIL", "")
    cur_modalidade_ensino = row.get("Modalidade de Ensino", "")
    
    if cur_id not in seen_cursos:
        cursos.append((cur_id, None, cur_nome, cur_cod_area, cur_nome_area, cur_modalidade_ensino))
        seen_cursos.add(cur_id)

sql_cursos = """
INSERT INTO cursos (cur_id, cur_cod, cur_nome, cur_cod_area, cur_nome_area, cur_modalidade_ensino)
VALUES (%s, %s, %s, %s, %s, %s)
ON DUPLICATE KEY UPDATE 
    cur_nome = VALUES(cur_nome),
    cur_cod_area = VALUES(cur_cod_area),
    cur_nome_area = VALUES(cur_nome_area),
    cur_modalidade_ensino = VALUES(cur_modalidade_ensino);
"""
cursor.executemany(sql_cursos, cursos)

# === 4. Inserir fatos acadêmicos ===
valores = []

for i, row in df_acad.iterrows():
    try:
        tem_id = mapa_tempo[int(row["Ano de Referência"])]
    except KeyError:
        continue
        
    if pd.isna(row["Código da Unidade Federativa do Curso"]): # Universidades EAD, não servirá para nossos testes.
        continue
    
    valores.append((
        int(row["Código da Instituição"]),
        int(row["Código do Curso de Graduação"]),
        int(row["Código da Unidade Federativa do Curso"]),
        tem_id,
        int(row["Ano de Ingresso"]),
        int(row["Prazo de Integralização em Anos"]),
        int(row["Ano de Integralização do Curso"]),
        int(row["Prazo de Acompanhamento do Curso em anos"]),
        int(row["Ano Máximo de Acompanhamento do Curso"]),
        int(row.get("Quantidade de Ingressantes no Curso", 0)),
        int(row.get("Quantidade de Permanência no Curso no ano de referência", 0)),
        int(row.get("Quantidade de Concluintes no Curso no ano de referência", 0)),
        int(row.get("Quantidade de Desistência no Curso no ano de referência", 0)),
        int(row.get("Quantidade de Falecimentos no Curso no ano de referência", 0)),
        float(row.get("Taxa de Permanência - TAP", 0)),
        float(row.get("Taxa de Conclusão Acumulada - TCA", 0)),
        float(row.get("Taxa de Desistência Acumulada - TDA", 0)),
        float(row.get("Taxa de Conclusão Anual – TCAN", 0)),
        float(row.get("Taxa de Desistência Anual – TADA", 0))
    ))

sql_acad = """
INSERT INTO indicadores_academicos (
    uni_id, cur_id, uf_id, tem_id,
    ano_ingresso, prazo_integracao_anos, ano_integracao_curso,
    prazo_acompanhamento_anos, ano_maximo_acompanhamento,
    qt_ingressantes, qt_permanencia, qt_concluintes,
    qt_desistencia, qt_falecidos,
    taxa_permanencia, taxa_conclusao_acumulada, taxa_desistencia_acumulada,
    taxa_conclusao_anual, taxa_desistencia_anual
) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
"""

cursor.executemany(sql_acad, valores)
conn.commit()
cursor.close()
conn.close()